# Unit 2.1 - train baseline model 


In [1]:
# Environmental setup. Install required libraries. Create an environment with conda/anaconda with python 3.10.X
# In Windows, there is no tflite-runtime library (available on Linux, ARM, Android and other embedded systems)
!pip install "tensorflow==2.19.0"
!pip install "tf2onnx==1.16.1"
!pip install "onnx==1.16.2" "onnxruntime==1.18.1"
!pip install "numpy==1.26.4" matplotlib

  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.2.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached keras-3.12.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached namex-0.1.0-py3-none-any.whl.metadata (322 bytes)
   ---------------------------------------- 0.0/375.7 MB ? eta -:--:--
   ---------------------------------------- 1.8/375.7 MB 11.2 MB/s eta 0:00:34
   ---------------------------------------- 3.7/375.7 MB 9.5 MB/s eta 0:00:40
    --------------------------------------- 5.8/375.7 MB 9.8 MB/s eta 0

In [2]:
# imports and data loading

import os
import numpy as np
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

DATA_PATH = "uca_ehar_preprocessed_win100_step50.npz"
npz = np.load(DATA_PATH)

X_train = npz["X_train"]
y_train = npz["y_train"]
X_val   = npz["X_val"]
y_val   = npz["y_val"]
X_test  = npz["X_test"]
y_test  = npz["y_test"]
class_names = npz["class_names"]


# --- Normalize inputs: per-channel standardization ---

# Compute mean and std ONLY on the training set
train_mean = X_train.mean(axis=(0, 1), keepdims=True)   # shape (1, 1, 7)
train_std  = X_train.std(axis=(0, 1), keepdims=True)    # shape (1, 1, 7)

# Save it to a file
norm_stats_path = "har_norm_stats.npz"
np.savez(norm_stats_path, mean=train_mean, std=train_std)

# Avoid division by zero
train_std[train_std == 0] = 1.0

X_train_norm = (X_train - train_mean) / train_std
X_val_norm   = (X_val   - train_mean) / train_std
X_test_norm  = (X_test  - train_mean) / train_std

X_train = X_train_norm
X_val   = X_val_norm
X_test  = X_test_norm

print("Normalized X_train mean:", X_train.mean(), "std:", X_train.std())

print("y_train min/max:", y_train.min(), y_train.max())
print("Class distribution (train):", np.bincount(y_train))
print("Class distribution (test): ", np.bincount(y_test))
print("class_names:", class_names)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)
print("Classes:", class_names)



TensorFlow version: 2.19.0
Normalized X_train mean: -0.11823387 std: 0.99343055
y_train min/max: 0 7
Class distribution (train): [1091 1929 2215  493  426 1045  975   67]
Class distribution (test):  [251 409 455 112 103 244 180  13]
class_names: ['STANDING' 'SITTING' 'WALKING' 'WALKING_UPSTAIRS' 'WALKING_DOWNSTAIRS'
 'RUNNING' 'LYING' 'DRINKING']
Train: (8241, 100, 7) Val: (1766, 100, 7) Test: (1767, 100, 7)
Classes: ['STANDING' 'SITTING' 'WALKING' 'WALKING_UPSTAIRS' 'WALKING_DOWNSTAIRS'
 'RUNNING' 'LYING' 'DRINKING']


In [3]:
# Convert labels to one-hot vectors:
num_classes = len(class_names)

y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes)
y_val_cat   = tf.keras.utils.to_categorical(y_val, num_classes)
y_test_cat  = tf.keras.utils.to_categorical(y_test, num_classes)

input_shape = X_train.shape[1:]   # (window_size, num_channels), e.g. (100, 7)
print("Input shape:", input_shape, "Num classes:", num_classes)


Input shape: (100, 7) Num classes: 8


In [4]:
# Define a CNN baseline model

from tensorflow.keras import layers, models

def build_baseline_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv1D(64, kernel_size=5, padding="same", activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(64, kernel_size=5, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(pool_size=2)(x)
    x = layers.Dropout(0.3)(x)

    # Block 2
    x = layers.Conv1D(128, kernel_size=5, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling1D()(x)

    # Dense head
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=inputs, outputs=outputs, name="har_baseline_cnn")
    return model

model = build_baseline_model(input_shape, num_classes)
model.summary()



Model: "har_baseline_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100, 7)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 100, 64)        │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 100, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 100, 64)        │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 50, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 50, 128)        │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 50, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 82,504 (322.28 KB)

 Trainable params: 81,992 (320.28 KB)

 Non-trainable params: 512 (2.00 KB)

In [5]:
# compile

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)


In [6]:
# Train the model

callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
]

history = model.fit(
    X_train, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=40,          # a bit more
    batch_size=64,
    callbacks=callbacks,
    verbose=1,
)

Epoch 1/40
129/129 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.7329 - loss: 0.6719 - val_accuracy: 0.6200 - val_loss: 1.1484 - learning_rate: 0.0010
Epoch 2/40
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8231 - loss: 0.4165 - val_accuracy: 0.7191 - val_loss: 0.7119 - learning_rate: 0.0010
Epoch 3/40
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8511 - loss: 0.3585 - val_accuracy: 0.8567 - val_loss: 0.3493 - learning_rate: 0.0010
Epoch 4/40
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8654 - loss: 0.3131 - val_accuracy: 0.8766 - val_loss: 0.3000 - learning_rate: 0.0010
Epoch 5/40
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8765 - loss: 0.2945 - val_accuracy: 0.8754 - val_loss: 0.2829 - learning_rate: 0.0010
Epoch 6/40
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8802 - loss: 0.2765 - val_accuracy: 0.8811 - val_loss: 0.2642 - learning_rate: 0.0010
Epoch 7/40
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8873 - loss: 0.

In [7]:
# Evaluate on test set (We ideally want something in the 90–95% range to match the narrative in the unit)

test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
print(f"Baseline test accuracy: {test_acc*100:.2f}%")

Baseline test accuracy: 91.85%


In [8]:
# Save the Keras model (backup)


from pathlib import Path

MODELS_DIR = Path(".")
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# 1) Save Keras-format backup
keras_path = MODELS_DIR / "har_baseline.keras"
model.save(keras_path)  # SavedModel format
print("Saved Keras model to:", keras_path)

# 2) Export TensorFlow SavedModel (NO extension, use model.export)
saved_model_dir = MODELS_DIR / "har_baseline_savedmodel"

# IMPORTANT: use export, not save
model.export(saved_model_dir.as_posix())
print("Exported SavedModel to:", saved_model_dir)

Saved Keras model to: har_baseline.keras
INFO:tensorflow:Assets written to: har_baseline_savedmodel\assets


INFO:tensorflow:Assets written to: har_baseline_savedmodel\assets


Saved artifact at 'har_baseline_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 7), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2065018249184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065018243552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065018247424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065018250768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065018244080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065018244432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065018252000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065018252704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065018255872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065018256576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065018253056: TensorSpec(shape=

In [9]:
# Convert SavedModel → ONNX with tf2onnx

import tf2onnx
import tensorflow as tf
import onnx
from pathlib import Path
import os

MODELS_DIR = Path(".")
MODELS_DIR.mkdir(parents=True, exist_ok=True)

onnx_path = MODELS_DIR / "har_baseline.onnx"

spec = (tf.TensorSpec((None,) + input_shape, tf.float32, name="input"),)

onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=spec,
    opset=13,
)

onnx.save(onnx_model, onnx_path.as_posix())
print("Saved ONNX model to:", onnx_path)
print("Size on disk: {:.2f} KB".format(os.path.getsize(onnx_path) / 1024))



Saved ONNX model to: har_baseline.onnx
Size on disk: 327.19 KB


In [10]:
import onnxruntime as ort
import numpy as np

session = ort.InferenceSession(onnx_path.as_posix(), providers=["CPUExecutionProvider"])
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

sample = X_test[0:1].astype(np.float32)   # ya normalizado
out = session.run([output_name], {input_name: sample})[0]
pred_idx = out.argmax()
print("Pred:", class_names[pred_idx], "| True:", class_names[y_test[0]])


Pred: WALKING | True: WALKING
